# Create and validate FDO nanopublications

For the code below to work, you need to have your profile set up, specifying your ORCID identifier and generating a key pair, using this command:

    $ np setup

See also: https://nanopublication.github.io/nanopub-py/

### Importing the dependencies

In [92]:
import rdflib
from rdflib.namespace import RDF, SH, XSD
from nanopub import definitions, Nanopub, NanopubClient, load_profile, NanopubConf, FDONanopub
import datetime
from nanopub.namespaces import HDL, FDOF, PROV
from pyshacl import validate
from rdflib import Graph
import requests
import json

### Constructing the nanopublication

In [93]:
conf = NanopubConf(
    add_prov_generated_time=False,
    add_pubinfo_generated_time=True,
    attribute_assertion_to_profile=True,
    attribute_publication_to_profile=True,
    profile=load_profile(),
    use_test_server=True
)

# Example using URI 
fdo_uri = rdflib.URIRef(HDL["21.T11967/39b0ec87d17a4856c5f7"]) 

fdopub = FDONanopub(fdo_id=fdo_uri,
                    label="NumberFdo1",
                    conf=conf)

fdo_profile = '21.T11966/82045bd97a0acce88378'
fdo_profile_uri = rdflib.URIRef(HDL[fdo_profile])

# Adding attributes using URIs
fdopub.add_fdo_profile(rdflib.URIRef(HDL["21.T11966/82045bd97a0acce88378"]))
fdopub.add_fdo_data_ref(fdo_profile_uri)
fdopub.add_fdo_type('{"FdoGenre":"21.T11966/365ff9576c26ca6053db","FdoMimeType":"21.T11966/f919d9f152904f6c40db","FdoOperations":["21.T11966/1da6d8c42eb6a685a8b6"]}')
fdopub.add_fdo_status("created")
fdopub.add_fdo_service(rdflib.URIRef(HDL["21.T11967/service"]))

fdopub.assertion.add((fdo_uri, FDOF.hasMetadata, fdopub.metadata.np_uri))

# Example using handle
fdo_handle = "21.T11967/39b0ec87d17a4856c5f7" 

fdopub_handle = FDONanopub(fdo_id=fdo_handle,
                        label="NumberFdo2",
                        conf=conf)

# Adding attributes using handles
fdopub_handle.add_fdo_data_ref("21.T11967/83d2b3f39034b2ac78cd")
fdopub_handle.add_fdo_type('{"FdoGenre":"21.T11966/365ff9576c26ca6053db","FdoMimeType":"21.T11966/f919d9f152904f6c40db","FdoOperations":["21.T11966/1da6d8c42eb6a685a8b6"]}')
fdopub_handle.add_fdo_status("created")
fdopub_handle.add_fdo_service("21.T11967/service")

fdopub_handle.assertion.add((HDL[fdo_handle], FDOF.hasMetadata, fdopub_handle.metadata.np_uri))

orcid_uri = rdflib.URIRef(load_profile().orcid_id)
fdopub.provenance.add((fdopub.metadata.assertion, PROV.wasAttributedTo, orcid_uri))
fdopub.pubinfo.add((fdopub.metadata.np_uri, rdflib.DCTERMS.creator, orcid_uri))

print(fdopub)


@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://hdl.handle.net/21.T11966/> .
@prefix ns2: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> rdfs:label "FAIR Digital Object: NumberFdo1" ;
        dcterms:creator orcid:0009-0009-0118-9195 ;
        npx:introduces <https://hdl.handle.net/21.T11967/39b0ec87d17a4856c5f7> ;
        prov:generatedAtTime "2025-05-15T09:02:19.769859"^^xsd:dateTime ;
        prov:wasAttributedTo orcid:0009-0009-0118-9195 .

    ns1:06a6c27e3e2ef27779ec rdfs:label "DataRef" .

    ns1:06fae297d104953b2eaa rdfs:label "FdoType" .

    ns1:143d58e30d417a2cb75d rdfs:label "FdoStatus" .

    ns1:FdoPro

## Validate FDO nanopublication (experimental)

Note: This will eventually be moved to nanopub-py.

In [94]:
# construct handle URI for JSON response
profile_uri = "https://hdl.handle.net/api/handles/" + fdo_profile

print(profile_uri)

# get profile json
profile_response = requests.get(profile_uri)
profile_data = profile_response.json()
print(json.dumps(profile_data, indent=2))

https://hdl.handle.net/api/handles/21.T11966/82045bd97a0acce88378
{
  "responseCode": 1,
  "handle": "21.T11966/82045bd97a0acce88378",
  "values": [
    {
      "index": 100,
      "type": "HS_ADMIN",
      "data": {
        "format": "admin",
        "value": {
          "handle": "0.NA/21.T11966",
          "index": 300,
          "permissions": "111111111111"
        }
      },
      "ttl": 86400,
      "timestamp": "2025-03-24T12:45:31Z"
    },
    {
      "index": 1,
      "type": "10320/loc",
      "data": {
        "format": "string",
        "value": "<locations>\n<location href=\"http://typeregistry.testbed.pid.gwdg.de/objects/21.T11966/82045bd97a0acce88378\" weight=\"0\" view=\"json\" />\n<location href=\"http://typeregistry.testbed.pid.gwdg.de/#objects/21.T11966/82045bd97a0acce88378\" weight=\"1\" view=\"ui\" />\n</locations>"
      },
      "ttl": 86400,
      "timestamp": "2025-03-24T12:45:31Z"
    },
    {
      "index": 2,
      "type": "21.T11966/FdoProfile",
      "dat

In [95]:
# find JSON schema from the profile
jsonschema_entry = next(
    (v for v in profile_data.get("values", []) if v.get("type") == "21.T11966/JsonSchema"),
    None
)

if jsonschema_entry:
    raw_value = jsonschema_entry["data"]["value"]
    parsed_value = json.loads(raw_value)
    jsonschema_url = parsed_value.get("$ref")
    print("JSON Schema URL:", jsonschema_url)
else:
    print("JsonSchema type not found.")
    
# fetch JSON schema
schema_url = jsonschema_url  

response = requests.get(schema_url)
json_schema = response.json()

print(json.dumps(json_schema, indent=2))

JSON Schema URL: https://typeapi.lab.pidconsortium.net/v1/types/schema/21.T11966/82045bd97a0acce88378
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "@id": "hdl:21.T11966/82045bd97a0acce88378",
  "additionalProperties": true,
  "description": "The profile for an FDO that follows configuration type 4.",
  "properties": {
    "21.T11966/06a6c27e3e2ef27779ec": {
      "@id": "hdl:21.T11966/06a6c27e3e2ef27779ec",
      "pattern": "^([0-9,A-Z,a-z])+(\\.([0-9,A-Z,a-z])+)*\\/([!-~])+$",
      "type": "string"
    },
    "21.T11966/143d58e30d417a2cb75d": {
      "@id": "hdl:21.T11966/143d58e30d417a2cb75d",
      "enum": [
        "created",
        "updated",
        "deleted",
        "moved",
        "embargoed"
      ],
      "type": "string"
    },
    "21.T11966/68763ca08f0783e44efa": {
      "@id": "hdl:21.T11966/68763ca08f0783e44efa",
      "pattern": "^([0-9,A-Z,a-z])+(\\.([0-9,A-Z,a-z])+)*\\/([!-~])+$",
      "type": "string"
    },
    "21.T11966/8b22ab68befd0e1b8e0c": {

In [99]:
# convert JSON schema to SHACL shape
EX = rdflib.Namespace("https://example.org/shapes")
HDL = rdflib.Namespace("https://hdl.handle.net/")

g = rdflib.Graph()
g.bind("sh", SH)
g.bind("xsd", XSD)
g.bind("ex", EX)
g.bind("hdl", HDL)

node_shape = EX["FdoProfileShape"]
g.add((node_shape, RDF.type, SH.NodeShape))
g.add((node_shape, SH.targetClass, rdflib.URIRef("https://w3id.org/fdof/ontology#SomeOtherClass")))
g.add((node_shape, SH.closed, rdflib.Literal(False)))

for field in json_schema["required"]:
    prop_shape = EX[field.replace("/", "_")]
    g.add((node_shape, SH.property, prop_shape))
    g.add((prop_shape, RDF.type, SH.PropertyShape))
    g.add((prop_shape, SH.path, URIRef(f"https://hdl.handle.net/{field}")))
    g.add((prop_shape, SH.minCount, Literal(1)))
    g.add((prop_shape, SH.maxCount, Literal(1)))
    g.add((prop_shape, SH.datatype, XSD.string))

shape = g

print(g.serialize(format="turtle"))


@prefix ex: <https://example.org/shapes> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:FdoProfileShape a sh:NodeShape ;
    sh:closed false ;
    sh:property ex:21.T11966_143d58e30d417a2cb75d,
        ex:21.T11966_FdoProfile,
        ex:21.T11966_b5b58656b1fa5aff0505 ;
    sh:targetClass <https://w3id.org/fdof/ontology#SomeOtherClass> .

ex:21.T11966_143d58e30d417a2cb75d a sh:PropertyShape ;
    sh:datatype xsd:string ;
    sh:maxCount 1 ;
    sh:minCount 1 ;
    sh:path <https://hdl.handle.net/21.T11966/143d58e30d417a2cb75d> .

ex:21.T11966_FdoProfile a sh:PropertyShape ;
    sh:datatype xsd:string ;
    sh:maxCount 1 ;
    sh:minCount 1 ;
    sh:path <https://hdl.handle.net/21.T11966/FdoProfile> .

ex:21.T11966_b5b58656b1fa5aff0505 a sh:PropertyShape ;
    sh:datatype xsd:string ;
    sh:maxCount 1 ;
    sh:minCount 1 ;
    sh:path <https://hdl.handle.net/21.T11966/b5b58656b1fa5aff0505> .




In [101]:
# validate assertion graph against shape graph

conforms, results_graph, results_text = validate(
    fdopub.assertion,
    shacl_graph=shape,
    inference='rdfs',
    abort_on_first=False,
    meta_shacl=False,
    advanced=True,
    debug=False
)

print(results_text)
print(results_graph.serialize(format="turtle"))

Validation Report
Conforms: True

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms true .




In [102]:
# Manually add attributes and labels - uncomment to make the nanopub invalid
fdopub.add_attribute(fdopub.FDO_STATUS_HANDLE, rdflib.Literal("published"))
fdopub.add_attribute_label(fdopub.FDO_STATUS_HANDLE, "FDO Status")

# should not be valid

conforms, results_graph, results_text = validate(
    fdopub.assertion,
    shacl_graph=shape,
    inference='rdfs',
    abort_on_first=False,
    meta_shacl=False,
    advanced=True,
    debug=False
)

print(results_text)
print(results_graph.serialize(format="turtle"))

Validation Report
Conforms: True

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms true .




### Signing the nanopublication

In [5]:
fdopub.sign()

In [6]:
print(fdopub)

Nanopub URI: https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://hdl.handle.net/21.T11966/> .
@prefix ns2: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w/> .
@prefix this: <https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:Head {
    this: a np:Nanopublication ;
        np:hasAssertion sub:assertion ;
        np:hasProvenance sub:provenance ;
        np:hasPublicationInfo sub:pubinfo .
}

sub:pubinfo {
    ns1:06a6c27e3e2ef27779ec rdfs:label "DataRef" .

    ns1:06fae297d104953b2eaa rdfs:label "FdoType" .

    ns1:143d58e30d417a2cb75d r

In [7]:
npuri = fdopub.metadata.np_uri
print(npuri)

https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w


### Publishing the nanopublication

In [8]:
fdopub.publish()

# Fetch again the published nanopublication from the network

In [9]:
fetchConf = NanopubConf(
    use_test_server=True
)
fetchNp = Nanopub(npuri, conf=fetchConf)

In [10]:
print(fetchNp)

Nanopub URI: https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://hdl.handle.net/21.T11966/> .
@prefix ns2: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w/> .
@prefix this: <https://w3id.org/np/RAj968NgOK6jZLNfmVLlDk_B27IUdEnOyTxIlRndrZE2w> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:pubinfo {
    ns1:06a6c27e3e2ef27779ec rdfs:label "DataRef" .

    ns1:06fae297d104953b2eaa rdfs:label "FdoType" .

    ns1:143d58e30d417a2cb75d rdfs:label "FDO Status",
            "FdoStatus" .

    ns1:b5b58656b1fa5aff0505 rdfs:label "FdoService" .

    sub:sig npx:hasAlgorithm "RSA" ;
        npx:hasPublicKey "MII